In [8]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

storm_df = pd.read_csv("/kaggle/input/noaa-powout-prism-0-1-is-storm-lag/noaapowoutprism_01_Is_Storm_Lag (1).csv")

In [9]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Define features
storm_features = [
    'DEATHS_INDIRECT', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
    'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'customers_out', 'duration_hours',
    'desc_word_count', 'desc_char_count',
    'has_tornado', 'has_hail', 'has_flood', 'has_wind', 'has_tree',
    'has_power', 'has_damage', 'has_outage', 'has_broken', 'has_blown',
    'tmin', 'tmax', 'tavg', 'ppt'
]

# Prepare X and y
X = storm_df[storm_features]
y = storm_df['is_storm_lagged'].astype(int)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 1st Model: Random Forest for classification whether a storm is occurs in the next hour or not

We use the Lag model approach to predict the whether a storm will occur in the next hour based on the given features. After testing a few models, we choose the Random Forest model for its  performance.

In [10]:
# Train Random Forest model on all data
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_scaled, y)

# Predict on all data
predictions = rf_model.predict(X_scaled)

# Create new DataFrame with predictions
storm_df_with_predictions = storm_df.copy()
storm_df_with_predictions['predicted_storm'] = predictions

In [11]:
storm_df_with_1_predictions = storm_df_with_predictions[storm_df_with_predictions['predicted_storm'] == 1].copy()

In [ ]:
storm_df_with_1_predictions

,Unnamed: 0,customers_out,power_outage_datetime,storm_start_datetime_est,storm_end_datetime_est,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,...,has_flood,has_wind,has_tree,has_power,has_damage,has_outage,has_broken,has_blown,is_storm_lagged,predicted_storm
4,25,1.0,2015-01-15 05:00:00,2015-01-15 05:00:00,2015-01-15 10:00:00,5.000000,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,0,1,1
5,26,1.0,2015-01-15 05:15:00,2015-01-15 05:00:00,2015-01-15 10:00:00,5.000000,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,0,1,1
6,27,1.0,2015-01-15 05:30:00,2015-01-15 05:30:00,2015-01-15 10:00:00,4.500000,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,0,1,1
7,28,1.0,2015-01-15 05:45:00,2015-01-15 05:30:00,2015-01-15 10:00:00,4.500000,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,0,1,1
8,29,1.0,2015-01-15 06:00:00,2015-01-15 06:00:00,2015-01-15 10:30:00,4.500000,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210264,395247,5.0,2023-12-25 20:00:00,2023-12-25 20:00:00,2023-12-26 00:46:00,4.766667,WYOMING,56.0,High Wind,Z,...,0,1,0,0,0,0,0,0,1,1
210265,395248,4.0,2023-12-25 20:15:00,2023-12-25 20:00:00,2023-12-26 00:46:00,4.766667,WYOMING,56.0,High Wind,Z,...,0,1,0,0,0,0,0,0,1,1
210269,395252,3.0,2023-12-25 21:30:00,2023-12-25 21:30:00,2023-12-26 16:45:00,19.250000,WYOMING,56.0,Heavy Snow,Z,...,0,1,0,0,0,0,0,0,1,1
210270,395253,3.0,2023-12-25 21:45:00,2023-12-25 21:36:00,2023-12-26 02:02:00,4.433333,WYOMING,56.0,Blizzard,Z,...,0,1,0,0,0,0,0,0,1,1


# 2nd Model: Random Forest for predicting severity of a storm if it occurs in the next hour

After Feature Selection, we selected the most relevant features for severity prediction after which we tested a few Classification Models after which the Random Forest model gave the best Performance (92% acc) to predict the severity of a storm if it occurs in the next hour. 

In [13]:
severity_features = [
    # Original impact features
    'DEATHS_INDIRECT', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
    'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'customers_out', 'duration_hours',

    # NLP-derived features
    'desc_word_count', 'desc_char_count',
    'has_tornado', 'has_hail', 'has_flood', 'has_wind', 'has_tree',
    'has_power', 'has_damage', 'has_outage', 'has_broken', 'has_blown',

    #prism features
    'tmin', 'tmax', 'tavg', 'ppt'
]

X = storm_df_with_1_predictions[severity_features]
y = storm_df_with_1_predictions['severity_class']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [14]:
# Train Random Forest model on all data
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_scaled, y)

# Predict on all data
predictions = rf_model.predict(X_scaled)

# Create new DataFrame with predictions
storm_data_with_severity = storm_df_with_1_predictions.copy()
storm_data_with_severity['severity_predicted'] = predictions

In [15]:
# Create three DataFrames based on predicted severity
df_low = storm_data_with_severity[storm_data_with_severity['severity_predicted'] == 0]
df_medium = storm_data_with_severity[storm_data_with_severity['severity_predicted'] == 1]
df_high = storm_data_with_severity[storm_data_with_severity['severity_predicted'] == 2]

In [16]:
df_low

,Unnamed: 0,customers_out,power_outage_datetime,storm_start_datetime_est,storm_end_datetime_est,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,...,has_wind,has_tree,has_power,has_damage,has_outage,has_broken,has_blown,is_storm_lagged,predicted_storm,severity_predicted
6,27,1.0,2015-01-15 05:30:00,2015-01-15 05:30:00,2015-01-15 10:00:00,4.5,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,1,1,0
7,28,1.0,2015-01-15 05:45:00,2015-01-15 05:30:00,2015-01-15 10:00:00,4.5,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,1,1,0
8,29,1.0,2015-01-15 06:00:00,2015-01-15 06:00:00,2015-01-15 10:30:00,4.5,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,1,1,0
9,30,1.0,2015-01-15 06:15:00,2015-01-15 06:00:00,2015-01-15 10:30:00,4.5,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,1,1,0
10,31,1.0,2015-01-15 06:30:00,2015-01-15 06:30:00,2015-01-15 13:00:00,6.5,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209935,394606,1.0,2023-03-24 23:30:00,2023-03-24 22:46:00,2023-03-25 01:40:00,2.9,WYOMING,56.0,High Wind,Z,...,1,0,0,0,0,0,0,1,1,0
209936,394607,2.0,2023-03-24 23:45:00,2023-03-24 22:46:00,2023-03-25 01:40:00,2.9,WYOMING,56.0,High Wind,Z,...,1,0,0,0,0,0,0,1,1,0
210102,394944,158.0,2023-09-04 13:00:00,2023-09-04 13:00:00,2023-09-04 14:00:00,1.0,WYOMING,56.0,High Wind,Z,...,1,0,0,0,0,0,0,1,1,0
210196,395126,1.0,2023-11-23 15:45:00,2023-11-23 15:05:00,2023-11-24 22:53:00,31.8,WYOMING,56.0,Heavy Snow,Z,...,0,0,0,0,0,0,0,1,1,0


In [17]:
df_medium

,Unnamed: 0,customers_out,power_outage_datetime,storm_start_datetime_est,storm_end_datetime_est,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,...,has_wind,has_tree,has_power,has_damage,has_outage,has_broken,has_blown,is_storm_lagged,predicted_storm,severity_predicted
4,25,1.0,2015-01-15 05:00:00,2015-01-15 05:00:00,2015-01-15 10:00:00,5.000000,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,1,1,1
5,26,1.0,2015-01-15 05:15:00,2015-01-15 05:00:00,2015-01-15 10:00:00,5.000000,ALABAMA,1.0,Winter Weather,Z,...,0,0,0,0,0,0,0,1,1,1
11,37,1.0,2015-01-25 16:30:00,2015-01-25 16:30:00,2015-01-25 16:40:00,0.166667,ALABAMA,1.0,Thunderstorm Wind,C,...,1,1,0,0,0,0,0,1,1,1
22,53,3.0,2015-02-20 19:00:00,2015-02-20 18:35:00,2015-02-21 03:15:00,8.666667,ALABAMA,1.0,Ice Storm,Z,...,0,0,0,0,0,0,0,1,1,1
24,59,287.0,2015-02-25 10:00:00,2015-02-25 10:00:00,2015-02-25 18:30:00,8.500000,ALABAMA,1.0,Heavy Snow,Z,...,0,0,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210252,395231,519.0,2023-12-23 15:00:00,2023-12-23 15:00:00,2023-12-24 03:00:00,12.000000,WYOMING,56.0,Winter Weather,Z,...,0,0,0,0,0,0,0,1,1,1
210256,395239,2.0,2023-12-25 18:00:00,2023-12-25 18:00:00,2023-12-26 16:00:00,22.000000,WYOMING,56.0,Winter Weather,Z,...,1,0,0,0,0,0,0,1,1,1
210257,395240,2.0,2023-12-25 18:15:00,2023-12-25 18:00:00,2023-12-26 16:00:00,22.000000,WYOMING,56.0,Winter Weather,Z,...,1,0,0,0,0,0,0,1,1,1
210258,395241,2.0,2023-12-25 18:30:00,2023-12-25 18:00:00,2023-12-26 16:00:00,22.000000,WYOMING,56.0,Winter Weather,Z,...,1,0,0,0,0,0,0,1,1,1


In [18]:
df_high

,Unnamed: 0,customers_out,power_outage_datetime,storm_start_datetime_est,storm_end_datetime_est,duration_hours,STATE,STATE_FIPS,EVENT_TYPE,CZ_TYPE,...,has_wind,has_tree,has_power,has_damage,has_outage,has_broken,has_blown,is_storm_lagged,predicted_storm,severity_predicted
23,54,79.0,2015-02-20 20:45:00,2015-02-20 20:40:00,2015-02-21 03:00:00,6.333333,ALABAMA,1.0,Ice Storm,Z,...,0,1,0,0,0,0,0,1,1,2
37,72,4.0,2015-02-25 14:00:00,2015-02-25 14:00:00,2015-02-25 20:30:00,6.500000,ALABAMA,1.0,Heavy Snow,Z,...,0,0,0,0,0,0,0,1,1,2
38,73,4.0,2015-02-25 14:15:00,2015-02-25 14:00:00,2015-02-25 20:30:00,6.500000,ALABAMA,1.0,Heavy Snow,Z,...,0,0,0,0,0,0,0,1,1,2
39,74,3.0,2015-02-25 14:30:00,2015-02-25 14:30:00,2015-02-25 21:30:00,7.000000,ALABAMA,1.0,Heavy Snow,Z,...,0,0,0,0,0,0,0,1,1,2
40,75,16.0,2015-02-25 14:45:00,2015-02-25 14:30:00,2015-02-25 21:30:00,7.000000,ALABAMA,1.0,Heavy Snow,Z,...,0,0,0,0,0,0,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210262,395245,3.0,2023-12-25 19:30:00,2023-12-25 19:00:00,2023-12-27 02:00:00,31.000000,WYOMING,56.0,Heavy Snow,Z,...,1,0,0,0,0,0,0,1,1,2
210263,395246,4.0,2023-12-25 19:45:00,2023-12-25 19:00:00,2023-12-27 02:00:00,31.000000,WYOMING,56.0,Heavy Snow,Z,...,1,0,0,0,0,0,0,1,1,2
210264,395247,5.0,2023-12-25 20:00:00,2023-12-25 20:00:00,2023-12-26 00:46:00,4.766667,WYOMING,56.0,High Wind,Z,...,1,0,0,0,0,0,0,1,1,2
210265,395248,4.0,2023-12-25 20:15:00,2023-12-25 20:00:00,2023-12-26 00:46:00,4.766667,WYOMING,56.0,High Wind,Z,...,1,0,0,0,0,0,0,1,1,2


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import lightgbm as lgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define outage features (common for all models)
outage_features = [
    'tmin', 'tmax', 'tavg', 'ppt',
    'has_tornado', 'has_hail', 'has_flood', 'has_wind', 'has_tree',
    'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'duration_hours',
    'desc_word_count', 'desc_char_count'
]

# Derive is_outage target (assuming customers_out > 0 indicates an outage)
for df in [df_low, df_medium, df_high]:
    df['is_outage'] = (df['customers_out'] > 0).astype(int)

/tmp/ipykernel_31/1656639219.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_outage'] = (df['customers_out'] > 0).astype(int)
/tmp/ipykernel_31/1656639219.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_outage'] = (df['customers_out'] > 0).astype(int)
/tmp/ipykernel_31/1656639219.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

# Handling the imbalanced data

After Going through the data, we found that the data is heavily imbalanced (our approach), where we had around 100 instances of No outage being true and 30000 of Outage being true . The problem was that there were a lot of instances where there was no storm, but an outage was reported. We didn't address this issue at the time, so to solve it we took all the non-storm rows and randomly assigned each of them to either of Low, Medium or High severity classes.

In [20]:
storm_no_outage_count_l = len(df_low[(df_low['predicted_storm'] == 1) & (df_low['customers_out'] == 0)])
storm_no_outage_count_l

152

In [21]:
storm_no_outage_count_m = len(df_medium[(df_medium['predicted_storm'] == 1) & (df_medium['customers_out'] == 0)])
storm_no_outage_count_m

82

In [22]:
storm_no_outage_count_h = len(df_high[(df_high['predicted_storm'] == 1) & (df_high['customers_out'] == 0)])
storm_no_outage_count_h

87

In [23]:
non_storm_no_outage_count = len(storm_df_with_predictions[(storm_df_with_predictions['predicted_storm'] == 0) & (storm_df_with_predictions['customers_out'] == 0)])
non_storm_no_outage_count

177

In [24]:
non_storm_count = len(storm_df_with_predictions[storm_df_with_predictions['predicted_storm'] == 0])
non_storm_count

103896

In [25]:
# Extract non-storm rows from storm_df_with_predictions
non_storm_data = storm_df_with_predictions[storm_df_with_predictions['predicted_storm'] == 0].copy()
non_storm_data['is_outage'] = 0  # No outage for non-storm events
non_storm_data['severity_predicted'] = 10  # Placeholder for non-storm rows

In [26]:
# Randomly split non-storm rows across the three DataFrames
non_storm_split = np.array_split(non_storm_data.sample(frac=1, random_state=42), 3)
non_storm_low, non_storm_medium, non_storm_high = non_storm_split

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [27]:
# Augment DataFrames with non-storm rows
df_low = pd.concat([df_low, non_storm_low], ignore_index=True)
df_medium = pd.concat([df_medium, non_storm_medium], ignore_index=True)
df_high = pd.concat([df_high, non_storm_high], ignore_index=True)

In [ ]:
df_low

# LOW

In [28]:
# 1. LightGBM for Low Severity
if not df_low.empty:
    X_low = df_low[outage_features]
    y_low = df_low['is_outage']
    
    # Train-test split
    X_train_low, X_test_low, y_train_low, y_test_low = train_test_split(
        X_low, y_low, test_size=0.2, stratify=y_low, random_state=42
    )
    
    # Scale features
    scaler_low = StandardScaler()
    X_train_low_scaled = scaler_low.fit_transform(X_train_low)
    X_test_low_scaled = scaler_low.transform(X_test_low)
    
    # Train LightGBM
    lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=42)
    lgb_model.fit(X_train_low_scaled, y_train_low)
    
    # Predict on test set
    low_predictions = lgb_model.predict(X_test_low_scaled)
    
    # Evaluate
    print("LightGBM (Low Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_low, low_predictions))
    print("Classification Report:\n", classification_report(y_test_low, low_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_low, low_predictions))

[LightGBM] [Info] Number of positive: 25787, number of negative: 27827
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004942 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2089
[LightGBM] [Info] Number of data points in the train set: 53614, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.480975 -> initscore=-0.076136
[LightGBM] [Info] Start training from score -0.076136
LightGBM (Low Severity) Metrics:
Accuracy: 0.9043569083855566
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.88      0.91      6957
           1       0.88      0.93      0.90      6447

    accuracy                           0.90     13404
   macro avg       0.90      0.91      0.90     13404
weighted avg       0.91      0.90      0.90     13404

Confusion Matrix:
 [[6125  83

In [29]:
from catboost import CatBoostClassifier

# 1. CatBoost for Low Severity
if not df_low.empty:
    X_low = df_low[outage_features]
    y_low = df_low['is_outage']
    
    # Train-test split
    X_train_low, X_test_low, y_train_low, y_test_low = train_test_split(
        X_low, y_low, test_size=0.2, stratify=y_low, random_state=42
    )
    
    # Scale features
    scaler_low = StandardScaler()
    X_train_low_scaled = scaler_low.fit_transform(X_train_low)
    X_test_low_scaled = scaler_low.transform(X_test_low)
    
    # Train CatBoost
    cat_model = CatBoostClassifier(
        iterations=100,
        learning_rate=0.1,
        depth=6,
        auto_class_weights='Balanced',
        random_state=42,
        verbose=0
    )
    cat_model.fit(X_train_low_scaled, y_train_low)
    
    # Predict on test set
    low_predictions = cat_model.predict(X_test_low_scaled)
    
    # Evaluate
    print("CatBoost (Low Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_low, low_predictions))
    print("Classification Report:\n", classification_report(y_test_low, low_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_low, low_predictions))

CatBoost (Low Severity) Metrics:
Accuracy: 0.8816025067144136
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.85      0.88      6957
           1       0.85      0.92      0.88      6447

    accuracy                           0.88     13404
   macro avg       0.88      0.88      0.88     13404
weighted avg       0.88      0.88      0.88     13404

Confusion Matrix:
 [[5907 1050]
 [ 537 5910]]


In [30]:
# 2. Random Forest for Low Severity
if not df_medium.empty:
    X_medium = df_low[outage_features]
    y_medium = df_low['is_outage']
    
    # Train-test split
    X_train_medium, X_test_medium, y_train_medium, y_test_medium = train_test_split(
        X_medium, y_medium, test_size=0.2, stratify=y_medium, random_state=42
    )
    
    # Scale features
    scaler_medium = StandardScaler()
    X_train_medium_scaled = scaler_medium.fit_transform(X_train_medium)
    X_test_medium_scaled = scaler_medium.transform(X_test_medium)
    
    # Train Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train_medium_scaled, y_train_medium)
    
    # Predict on test set
    medium_predictions = rf_model.predict(X_test_medium_scaled)
    
    # Evaluate
    print("\nRandom Forest (Low Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_medium, medium_predictions))
    print("Classification Report:\n", classification_report(y_test_medium, medium_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_medium, medium_predictions))


Random Forest (Low Severity) Metrics:
Accuracy: 0.9530737093404954
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.95      0.95      6957
           1       0.95      0.96      0.95      6447

    accuracy                           0.95     13404
   macro avg       0.95      0.95      0.95     13404
weighted avg       0.95      0.95      0.95     13404

Confusion Matrix:
 [[6612  345]
 [ 284 6163]]


# MEDIUM

In [31]:
# 2. Random Forest for Medium Severity
if not df_medium.empty:
    X_medium = df_medium[outage_features]
    y_medium = df_medium['is_outage']
    
    # Train-test split
    X_train_medium, X_test_medium, y_train_medium, y_test_medium = train_test_split(
        X_medium, y_medium, test_size=0.2, stratify=y_medium, random_state=42
    )
    
    # Scale features
    scaler_medium = StandardScaler()
    X_train_medium_scaled = scaler_medium.fit_transform(X_train_medium)
    X_test_medium_scaled = scaler_medium.transform(X_test_medium)
    
    # Train Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train_medium_scaled, y_train_medium)
    
    # Predict on test set
    medium_predictions = rf_model.predict(X_test_medium_scaled)
    
    # Evaluate
    print("\nRandom Forest (Medium Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_medium, medium_predictions))
    print("Classification Report:\n", classification_report(y_test_medium, medium_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_medium, medium_predictions))


Random Forest (Medium Severity) Metrics:
Accuracy: 0.9278524779101037
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.93      6943
           1       0.91      0.94      0.92      6072

    accuracy                           0.93     13015
   macro avg       0.93      0.93      0.93     13015
weighted avg       0.93      0.93      0.93     13015

Confusion Matrix:
 [[6372  571]
 [ 368 5704]]


# HIGH

In [32]:
# 3. LSTM for High Severity
if not df_high.empty:
    # Function to create sequences for LSTM
    def create_sequences(data, seq_length=3):
        sequences = []
        targets = []
        indices = []
        for state in data['STATE_FIPS'].unique():
            state_data = data[data['STATE_FIPS'] == state].sort_values('power_outage_datetime')
            X_state = state_data[outage_features].values
            y_state = state_data['is_outage'].values
            state_indices = state_data.index
            for i in range(len(X_state) - seq_length + 1):
                sequences.append(X_state[i:i + seq_length])
                targets.append(y_state[i + seq_length - 1])
                indices.append(state_indices[i + seq_length - 1])
        return np.array(sequences), np.array(targets), indices
    
    # Prepare sequences
    seq_length = 3
    X_high, y_high, high_indices = create_sequences(df_high, seq_length)
    
    # Train-test split for sequences
    X_train_high, X_test_high, y_train_high, y_test_high, train_indices, test_indices = train_test_split(
        X_high, y_high, high_indices, test_size=0.2, stratify=y_high, random_state=42
    )
    
    # Scale features
    scaler_high = StandardScaler()
    X_train_high_scaled = np.array([scaler_high.fit_transform(seq) for seq in X_train_high])
    X_test_high_scaled = np.array([scaler_high.transform(seq) for seq in X_test_high])
    
    # Define LSTM model
    lstm_model = Sequential([
        LSTM(50, input_shape=(seq_length, len(outage_features)), return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    lstm_model.compile(optimizer='adam', loss='binary_crossentropy')
    
    # Train LSTM
    lstm_model.fit(X_train_high_scaled, y_train_high, epochs=10, batch_size=32, verbose=0)
    
    # Predict on test set
    high_predictions = (lstm_model.predict(X_test_high_scaled) > 0.5).astype(int).flatten()
    
    # Evaluate
    print("\nLSTM (High Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_high, high_predictions))
    print("Classification Report:\n", classification_report(y_test_high, high_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_high, high_predictions))

I0000 00:00:1745733609.703601      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1745733613.430093     148 cuda_dnn.cc:529] Loaded cuDNN version 90300


418/418 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

LSTM (High Severity) Metrics:
Accuracy: 0.5205633380777587
Classification Report:
               precision    recall  f1-score   support

           0       0.52      1.00      0.68      6936
           1       0.70      0.00      0.01      6413

    accuracy                           0.52     13349
   macro avg       0.61      0.50      0.35     13349
weighted avg       0.61      0.52      0.36     13349

Confusion Matrix:
 [[6926   10]
 [6390   23]]


In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [34]:
# 3. Feedforward Neural Network for High Severity
if not df_high.empty:
    X_high = df_high[outage_features]
    y_high = df_high['is_outage']
    
    # Train-test split
    X_train_high, X_test_high, y_train_high, y_test_high = train_test_split(
        X_high, y_high, test_size=0.2, stratify=y_high, random_state=42
    )
    
    # Scale features
    scaler_high = StandardScaler()
    X_train_high_scaled = scaler_high.fit_transform(X_train_high)
    X_test_high_scaled = scaler_high.transform(X_test_high)
    
    # Define FNN model
    fnn_model = Sequential([
        Dense(64, activation='relu', input_shape=(len(outage_features),)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    # Compile model
    fnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train model
    fnn_model.fit(
        X_train_high_scaled, y_train_high,
        epochs=20,
        batch_size=64,
        validation_split=0.2,
        verbose=0
    )
    
    # Predict on test set
    high_predictions = (fnn_model.predict(X_test_high_scaled) > 0.5).astype(int).flatten()
    
    # Evaluate
    print("\nFeedforward Neural Network (High Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_high, high_predictions))
    print("Classification Report:\n", classification_report(y_test_high, high_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_high, high_predictions))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1745733666.603021     147 service.cc:148] XLA service 0x7ded7c010100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745733666.603598     147 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1745733668.179604     147 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


418/418 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

Feedforward Neural Network (High Severity) Metrics:
Accuracy: 0.8706612806702573
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.85      0.87      6944
           1       0.85      0.89      0.87      6424

    accuracy                           0.87     13368
   macro avg       0.87      0.87      0.87     13368
weighted avg       0.87      0.87      0.87     13368

Confusion Matrix:
 [[5919 1025]
 [ 704 5720]]


In [35]:
# 2. Random Forest for High Severity
if not df_high.empty:
    X_medium = df_high[outage_features]
    y_medium = df_high['is_outage']
    
    # Train-test split
    X_train_medium, X_test_medium, y_train_medium, y_test_medium = train_test_split(
        X_medium, y_medium, test_size=0.2, stratify=y_medium, random_state=42
    )
    
    # Scale features
    scaler_medium = StandardScaler()
    X_train_medium_scaled = scaler_medium.fit_transform(X_train_medium)
    X_test_medium_scaled = scaler_medium.transform(X_test_medium)
    
    # Train Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train_medium_scaled, y_train_medium)
    
    # Predict on test set
    medium_predictions = rf_model.predict(X_test_medium_scaled)
    
    # Evaluate
    print("\nRandom Forest (High Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_medium, medium_predictions))
    print("Classification Report:\n", classification_report(y_test_medium, medium_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_medium, medium_predictions))


Random Forest (High Severity) Metrics:
Accuracy: 0.9367145421903053
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94      6944
           1       0.92      0.95      0.94      6424

    accuracy                           0.94     13368
   macro avg       0.94      0.94      0.94     13368
weighted avg       0.94      0.94      0.94     13368

Confusion Matrix:
 [[6425  519]
 [ 327 6097]]


Feature Scaling

In [31]:
from sklearn.model_selection import RandomizedSearchCV

# Parameter grid
rf_param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

# Initialize Random Forest
rf_estimator = RandomForestClassifier(random_state=42, n_jobs=-1)

# RandomizedSearchCV
rf_random_search = RandomizedSearchCV(
    estimator=rf_estimator,
    param_distributions=rf_param_grid,
    n_iter=30,  # number of random combinations to try
    scoring='f1',  # or 'accuracy' if you prefer
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit to training data
rf_random_search.fit(X_train_medium_scaled, y_train_medium)

# Best model
best_rf_model = rf_random_search.best_estimator_

# Predict and evaluate
best_rf_predictions = best_rf_model.predict(X_test_medium_scaled)

print("\nTuned Random Forest (Low Severity) Metrics:")
print("Accuracy:", accuracy_score(y_test_medium, best_rf_predictions))
print("Classification Report:\n", classification_report(y_test_medium, best_rf_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test_medium, best_rf_predictions))

# Show best parameters
print("Best Parameters:\n", rf_random_search.best_params_)


Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=  15.8s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=  19.6s
[CV] END bootstrap=True, max_depth=20, max_features=None, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time= 2.3min
[CV] END bootstrap=True, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   7.4s
[CV] END bootstrap=True, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   7.8s
[CV] END bootstrap=True, max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   7.8s
[CV] END bootstrap=False, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=500; tota

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



Tuned Random Forest (Low Severity) Metrics:
Accuracy: 0.937013764213046
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94      6944
           1       0.92      0.95      0.94      6424

    accuracy                           0.94     13368
   macro avg       0.94      0.94      0.94     13368
weighted avg       0.94      0.94      0.94     13368

Confusion Matrix:
 [[6428  516]
 [ 326 6098]]
Best Parameters:
 {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 50, 'bootstrap': True}


In [32]:
from sklearn.model_selection import RandomizedSearchCV

# Parameter grid for Random Forest
rf_param_grid_medium = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

# Initialize Random Forest
rf_estimator_medium = RandomForestClassifier(random_state=42, n_jobs=-1)

# RandomizedSearchCV setup
rf_random_search_medium = RandomizedSearchCV(
    estimator=rf_estimator_medium,
    param_distributions=rf_param_grid_medium,
    n_iter=30,  # Number of random combinations to try
    scoring='f1',  # You can switch to 'accuracy' if you prefer
    cv=3,  # 3-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit RandomizedSearchCV to training data
rf_random_search_medium.fit(X_train_medium_scaled, y_train_medium)

# Best Random Forest model
best_rf_model_medium = rf_random_search_medium.best_estimator_

# Predict and evaluate
best_medium_predictions = best_rf_model_medium.predict(X_test_medium_scaled)

print("\nTuned Random Forest (Medium Severity) Metrics:")
print("Accuracy:", accuracy_score(y_test_medium, best_medium_predictions))
print("Classification Report:\n", classification_report(y_test_medium, best_medium_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test_medium, best_medium_predictions))

# Best parameters found
print("Best Parameters:\n", rf_random_search_medium.best_params_)


Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=  16.3s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=  19.7s
[CV] END bootstrap=True, max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=500; total time=  36.0s
[CV] END bootstrap=True, max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=500; total time=  35.8s
[CV] END bootstrap=True, max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=500; total time=  37.3s
[CV] END bootstrap=False, max_depth=10, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=  14.1s
[CV] END bootstrap=False, max_depth=50, max_features=None, min_samples_leaf=2, min_samples_split=5, n_estimators=200; to

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(



Tuned Random Forest (Medium Severity) Metrics:
Accuracy: 0.937013764213046
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94      6944
           1       0.92      0.95      0.94      6424

    accuracy                           0.94     13368
   macro avg       0.94      0.94      0.94     13368
weighted avg       0.94      0.94      0.94     13368

Confusion Matrix:
 [[6428  516]
 [ 326 6098]]
Best Parameters:
 {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 50, 'bootstrap': True}


In [37]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Function to create model (for KerasClassifier)
def create_fnn_model(hidden_layer_sizes=(64, 32, 16), dropout_rate=0.3, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(hidden_layer_sizes[0], activation='relu', input_shape=(len(outage_features),)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(hidden_layer_sizes[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(hidden_layer_sizes[2], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Wrap model
fnn_wrapper = KerasClassifier(
    model=create_fnn_model,
    verbose=0
)

# Define parameter grid
fnn_param_grid = {
    'model__hidden_layer_sizes': [(128, 64, 32), (64, 32, 16), (32, 16, 8)],
    'model__dropout_rate': [0.2, 0.3, 0.5],
    'model__learning_rate': [1e-2, 1e-3, 1e-4],
    'batch_size': [32, 64, 128],
    'epochs': [30, 50, 70]
}

# RandomizedSearchCV
fnn_random_search = RandomizedSearchCV(
    estimator=fnn_wrapper,
    param_distributions=fnn_param_grid,
    n_iter=20,  # number of random combinations
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit RandomizedSearchCV
fnn_random_search.fit(X_train_high_scaled, y_train_high)

# Best model
best_fnn_model = fnn_random_search.best_estimator_

# Predict and evaluate
high_predictions_best = (best_fnn_model.predict(X_test_high_scaled) > 0.5).astype(int).flatten()

print("\nTuned Feedforward Neural Network (High Severity) Metrics:")
print("Accuracy:", accuracy_score(y_test_high, high_predictions_best))
print("Classification Report:\n", classification_report(y_test_high, high_predictions_best))
print("Confusion Matrix:\n", confusion_matrix(y_test_high, high_predictions_best))

# Best parameters
print("Best Parameters:\n", fnn_random_search.best_params_)


ModuleNotFoundError: No module named 'scikeras'

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib  # For saving model

# 1. Prepare data
if not df_low.empty:
    X_low = df_low[outage_features]
    y_low = df_low['is_outage']
    
    # Train-test split
    X_train_low, X_test_low, y_train_low, y_test_low = train_test_split(
        X_low, y_low, test_size=0.2, stratify=y_low, random_state=42
    )

    # 2. Train Random Forest (no scaling needed!)
    rf_final_model = RandomForestClassifier(
        n_estimators=300,    # Slightly more trees for stability
        max_depth=None,      # Let trees grow fully unless needed
        random_state=42,
        n_jobs=-1,
        bootstrap=True
    )
    rf_final_model.fit(X_train_low, y_train_low)

    # 3. Evaluate
    low_predictions = rf_final_model.predict(X_test_low)
    
    print("\nFinal Random Forest (Low Severity) Metrics:")
    print("Accuracy:", accuracy_score(y_test_low, low_predictions))
    print("Classification Report:\n", classification_report(y_test_low, low_predictions))
    print("Confusion Matrix:\n", confusion_matrix(y_test_low, low_predictions))

    # 4. Save model
    joblib.dump(rf_final_model, 'rf_low_severity_model.pkl')
    print("\nModel saved as 'rf_low_severity_model.pkl'")



Final Random Forest (Low Severity) Metrics:
Accuracy: 0.9532975231274247
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.95      0.95      6957
           1       0.95      0.96      0.95      6447

    accuracy                           0.95     13404
   macro avg       0.95      0.95      0.95     13404
weighted avg       0.95      0.95      0.95     13404

Confusion Matrix:
 [[6611  346]
 [ 280 6167]]

Model saved as 'rf_low_severity_model.pkl'


In [2]:
pip install scikeras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 103.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
